In [1]:
import numpy as np
import pandas as pd
import sklearn.linear_model as lm
import sklearn.preprocessing as pre
import sklearn.model_selection as ms
import matplotlib.pyplot as plt
import seaborn as sns

import harness

In [2]:
flights_train = harness.read_flights('data/flights_train.csv')
flights_test = harness.read_flights('data/flights_test.csv')

In [3]:
flights_clean = harness.clean_train(flights_train)

In [4]:
add_features = harness.chain(
    harness.add_date_parts, harness.add_haul,
    harness.add_hour,
)

In [5]:
flights_with_features = add_features(flights_clean)

In [6]:
x_transform = harness.chain(
    harness.keep_only_test_columns,
    add_features,
    [harness.make_all_dummies, flights_with_features],
    harness.only_numeric,
    harness.scale,
)

In [7]:
transformer = harness.DataTransformer(
    x_transform, harness.remove_early
)

In [8]:
x_train, y_train = transformer.extract_transform(flights_clean)

In [9]:
def train_model():
    linreg = lm.Ridge(random_state=42)
    grid_search = ms.GridSearchCV(
        linreg, dict(alpha=[10 ** i for i in range(-6, 7)]),
        scoring=transformer.score_transformed,
        njobs=-1
    )
    grid_search.fit(x_train.values, y_train.values)
    harness.save(grid_search.best_estimator_, 'everything_linear_model')

In [10]:
# train_model()

In [11]:
linreg = harness.load('everything_linear_model')
linreg_model = harness.TrainedModel(linreg, transformer)

In [12]:
x_test, y_test = transformer.extract_transform(flights_test)

In [13]:
linreg_model.validate(harness.clean_train(flights_test))

R squared: 0.00242
Median absolute error: 22.4
R squared (no early): 0.0246
Median absolute error (no early): 12.4

In [14]:
linreg_model.submit(
    'data/test.csv', 'everything_linear_submission.csv', 'predicted_delay'
)